# Getting started with MillionTrees datasets

In [3]:
import milliontrees
from torchvision import transforms

# List available datasets
print(milliontrees.benchmark_datasets)

['iwildcam']


The general workflow is to 
1. Select and optionally download a dataset
2. Load the train and test splits from the dataset
3. Create a torchvision transform to instruct the dataloader how to preprocess images, and optionally augment, input images and metadata
4. Create a pytorch dataloader
4. Use these dataloaders to train models in native pytorch or pytorch lightning

Select and optionally download a dataset

In [ ]:
# Load the point dataset
dataset = milliontrees.datasets.get_dataset("TreePoints")

Datasets are split into groups of images based on task. For example, 'train' versus 'test' or 'zero_shot_train' and 'zero_shot_test'.

In [2]:
dataset.list_subsets()
train_data = dataset.get_subset(
    "train",
    transform=transforms.Compose(
        [transforms.Resize((448, 448)), transforms.ToTensor()]
    ),
)

NameError: name 'dataset' is not defined